<a href="https://colab.research.google.com/github/friederrr/proof_contamination/blob/main/code/CDM_eval/CD_pipeline_CV_ngram_acc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CD PIPELINE (mit CV) n-gram accuracy

In [ ]:
%%capture
!pip install datasets

In [ ]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import random
import numpy as np
import pandas as pd
import copy
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import torch
import json
from datetime import datetime
import itertools
from itertools import cycle, product
import re
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from pathlib import Path
import statistics
from sklearn.model_selection import StratifiedKFold

In [ ]:
from CDMs_functions_v1 import *

## Model Selektion

**OLMO**:

- "allenai/OLMo-7B-0724-hf": Downlaod (F32): (27 GB), GPU-RAM (bfloat16): 14 GB (T4 sufficient)

- "allenai/OLMo-7B-0724-SFT-hf": Download (BF16) (14 GB), GPU-RAM (14 GB)

- "allenai/OLMo-7B-0724-Instruct-hf": Download (BF16) (14 GB), GPU-RAM (14 GB)

- "allenai/OLMo-2-1124-13B-Instruct": Download (BF16) (28 GB), GPU-RAM (27 GB) -> works fine with A100!

- "allenai/OLMo-2-0325-32B-Instruct": Download (BF16) (65 GB), GPU-RAM (34 GB). It needs about 15min to answer one question with A100 (meta-device offloading). However, besides from that, the pipeline was compatible. Though, it is not recommended to use it with A100.

**DEEPSEEK**:
- "deepseek-ai/deepseek-math-7b-instruct":

**LEMMA**:
- "EleutherAI/llemma_7b"

In [ ]:
#!Parameter
model_id = "allenai/OLMo-2-1124-13B-Instruct"
DRIVE_PATH=Path('/content/drive/MyDrive/Masterarbeit25/')

In [ ]:
dir_token_path=DRIVE_PATH / "MathCONTA_ngram_generation_data"
file_paths=list_files_in_directory(dir_token_path, model_id)

Files in /content/drive/MyDrive/Masterarbeit25/MathCONTA_ngram_generation_data/allenai/OLMo-2-1124-13B-Instruct:

OLMo-2-1124-13B-Instruct/Forum_full_temp0_nstarts20_n1_2_3_5_7_seed42.json
OLMo-2-1124-13B-Instruct/AMC8_full_temp0_nstarts20_n1_2_3_5_7_seed42.json
OLMo-2-1124-13B-Instruct/AIME_full_temp0_nstarts20_n1_2_3_5_7_seed42.json
OLMo-2-1124-13B-Instruct/word-problems_full_temp0_nstarts20_n1_2_3_5_7_seed42.json
OLMo-2-1124-13B-Instruct/MathCONTA_full_temp0_nstarts20_n1_2_3_5_7_seed42.json
OLMo-2-1124-13B-Instruct/MathCONTA_train_full_temp0_nstarts20_n1_2_3_5_7_seed42.json
OLMo-2-1124-13B-Instruct/MathCONTA_test_full_temp0_nstarts20_n1_2_3_5_7_seed42.json


In [ ]:
#merge_json_files(file_paths, dir_token_path / model_id / "MathCONTA_full_temp0_nstarts20_n1_2_3_5_7_seed42.json")

In [ ]:
#if file exists you don't have to load the models
load_models=False
if load_models:
  model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map='auto')
  tokenizer = AutoTokenizer.from_pretrained(model_id)
else:
  model = None
  tokenizer = None
  ds_conta = None

## DATA Processing/Loading (Standard)

In [ ]:
#!Parameter
repo_id = "Tobstar001/MathCONTA"
split = "test"
config_name="core"

ds_conta = load_dataset(path=repo_id,name=config_name,split=split)

In [ ]:
#filter ds_conta
#ds_conta_w = ds_conta.filter(lambda example: example["CATEGORY"] == "word-problems")
ds_conta_ai = ds_conta.filter(lambda example: example["CATEGORY"] == "AIME")
ds_conta_am = ds_conta.filter(lambda example: example["CATEGORY"] == "AMC8")
ds_conta_f = ds_conta.filter(lambda example: example["CATEGORY"] == "Forum")

In [ ]:
data_name="MathCONTA_full_temp0" #please add full|problem and tempx to the name: e.g. AIME_full_temp0
n_starts=20
n_range=[1,2,3,5,7]
temp=0.0
torch_seed=42
seed_rand=42
start_offset=5
random_starts=True
only_problem=False
force_reprocess=False
verbose=False
data=ds_conta

MathCONTA_ngram_data=create_ngram_generation_data(model_id=model_id,ds_conta=data,
                                                 model=model, tokenizer=tokenizer, data_name=data_name,
                                                force_reprocess=force_reprocess,n_starts=n_starts,n=n_range,temp=temp,
                                                 torch_seed=torch_seed,seed_rand=seed_rand,start_offset=start_offset,random_starts=random_starts,
                                                 only_problem=only_problem,verbose=verbose,
                                                 base_path_token=dir_token_path)

File path: /content/drive/MyDrive/Masterarbeit25/MathCONTA_ngram_generation_data/allenai/OLMo-2-1124-13B-Instruct/MathCONTA_full_temp0_nstarts20_n1_2_3_5_7_seed42.json
Loading existing n-gram generation data from: /content/drive/MyDrive/Masterarbeit25/MathCONTA_ngram_generation_data/allenai/OLMo-2-1124-13B-Instruct/MathCONTA_full_temp0_nstarts20_n1_2_3_5_7_seed42.json


In [ ]:
ngram_df=get_ngram_df_from_generation_data(MathCONTA_ngram_data, n=2)
ngram_df

,ID,CATEGORY,LABEL,LABEL_BINARY,n,ngram_acc
0,owm-word-1,word-problems,Conta,1,2,0.30
1,clean-word-1,word-problems,Clean,0,2,0.50
2,owm-word-2,word-problems,Conta,1,2,0.15
3,clean-word-2,word-problems,Clean,0,2,0.40
4,owm-word-3,word-problems,Conta,1,2,0.35
...,...,...,...,...,...,...
95,clean-forum-6,Forum,Clean,0,2,0.45
96,clean-forum-7,Forum,Clean,0,2,0.50
97,clean-forum-8,Forum,Clean,0,2,0.50
98,clean-forum-9,Forum,Clean,0,2,0.75


## Train Test Split (Only relevant once)
Subsequent Stratified Train test split for google drive
For new data please filter train/test before generating the data

In [ ]:
train, test = stratified_dict_split(MathCONTA_ngram_data, test_ratio=0.3, seed=42)

Train size: 70, Test size: 30
{'owm-aime-4', 'owm-forum-10', 'owm-amc8-12', 'clean-forum-6', 'owm-amc8-9', 'clean-forum-7', 'owm-forum-5', 'clean-word-12', 'owm-word-10', 'clean-word-7', 'clean-aime-3', 'owm-amc8-15', 'owm-aime-7', 'clean-amc8-10', 'owm-word-5', 'clean-amc8-3', 'clean-word-10', 'clean-amc8-14', 'owm-word-2', 'owm-amc8-10', 'owm-word-9', 'owm-forum-3', 'clean-aime-4', 'clean-amc8-18', 'clean-aime-7', 'clean-amc8-1', 'clean-word-9', 'owm-amc8-17', 'clean-forum-9', 'owm-aime-3'}


In [ ]:
#train_path: dir_token_path / model_id / "MathCONTA_train_sample50_max200_seed42.json", test path accordingly
file_path_train=dir_token_path / model_id / "MathCONTA_train_full_temp0_nstarts20_n1_2_3_5_7_seed42.json"
with open(file_path_train, 'w') as f:
      json.dump(train, f)

file_path_test=dir_token_path / model_id / "MathCONTA_test_full_temp0_nstarts20_n1_2_3_5_7_seed42.json"
with open(file_path_test, 'w') as f:
      json.dump(test, f)

## TRAIN SET CV - Parameter Tuning

In [ ]:
#how to filter on categories
#category='word-problems'
#MathCONTA_ngram_data_f = [entry for entry in MathCONTA_ngram_data if entry.get('CATEGORY') == category]
#len(MathCONTA_ngram_data_f)

24

In [ ]:
data_name="MathCONTA_train_full_temp0" #please add full|problem and tempx to the name: e.g. AIME_full_temp0
n_starts=20
n_range=[1,2,3,5,7]
temp=0.0
torch_seed=42
seed_rand=42
start_offset=5
random_starts=True
only_problem=False
force_reprocess=False
verbose=False
data=ds_conta

MathCONTA_ngram_data=create_ngram_generation_data(model_id=model_id,ds_conta=data,
                                                 model=model, tokenizer=tokenizer, data_name=data_name,
                                                force_reprocess=force_reprocess,n_starts=n_starts,n=n_range,temp=temp,
                                                 torch_seed=torch_seed,seed_rand=seed_rand,start_offset=start_offset,random_starts=random_starts,
                                                 only_problem=only_problem,verbose=verbose,
                                                 base_path_token=dir_token_path)

File path: /content/drive/MyDrive/Masterarbeit25/MathCONTA_ngram_generation_data/allenai/OLMo-2-1124-13B-Instruct/MathCONTA_train_full_temp0_nstarts20_n1_2_3_5_7_seed42.json
Loading existing n-gram generation data from: /content/drive/MyDrive/Masterarbeit25/MathCONTA_ngram_generation_data/allenai/OLMo-2-1124-13B-Instruct/MathCONTA_train_full_temp0_nstarts20_n1_2_3_5_7_seed42.json


In [ ]:
#!Parameter 2D
data=MathCONTA_ngram_data
feature_col="ngram_acc"
label_col="LABEL_BINARY"
n_range=n_range
cv_folds=5
cv_seed=42
log_path_base =DRIVE_PATH / "cdm_data" / "MathCONTA_v1"
exp_id="overleaf2"#+'_'+category
method_name="ngram_acc"


In [ ]:
out_dict=tune_ngram_acc(ngram_data=data,feature_col=feature_col,
                          label_col=label_col,cv_folds=cv_folds,cv_seed=cv_seed,
                          n_range=n_range, model_id=model_id,data_name=data_name,
                          method_name=method_name,log_path_base=log_path_base,exp_id=exp_id)

Testing n=1 | median_threshold=0.325000 | mean_cvacc_test=0.485714
Testing n=2 | median_threshold=0.525000 | mean_cvacc_test=0.457143
Testing n=3 | median_threshold=0.425000 | mean_cvacc_test=0.414286
Testing n=5 | median_threshold=0.375000 | mean_cvacc_test=0.442857
Testing n=7 | median_threshold=0.050000 | mean_cvacc_test=0.414286
Best log entry:
{'parameter': {'n': 1}, 'global_threshold': 0.325, 'median_threshold': 0.325, 'mean_cvacc_train': 0.521429, 'mean_cvacc_test': 0.485714}
Accuracy log saved in /content/drive/MyDrive/Masterarbeit25/cdm_data/MathCONTA_v1/allenai/OLMo-2-1124-13B-Instruct/ngram_acc/MathCONTA_train_full_temp0_accuracylog_overleaf2.json


## TEST SET EVALUATION

In [ ]:
data_name="MathCONTA_test_full_temp0"
MathCONTA_ngram_data=create_ngram_generation_data(model_id=model_id,ds_conta=data,
                                                 model=model, tokenizer=tokenizer, data_name=data_name,
                                                force_reprocess=force_reprocess,n_starts=n_starts,n=n_range,temp=temp,
                                                 torch_seed=torch_seed,seed_rand=seed_rand,start_offset=start_offset,random_starts=random_starts,
                                                 only_problem=only_problem,verbose=verbose,
                                                 base_path_token=dir_token_path)

File path: /content/drive/MyDrive/Masterarbeit25/MathCONTA_ngram_generation_data/allenai/OLMo-2-1124-13B-Instruct/MathCONTA_test_full_temp0_nstarts20_n1_2_3_5_7_seed42.json
Loading existing n-gram generation data from: /content/drive/MyDrive/Masterarbeit25/MathCONTA_ngram_generation_data/allenai/OLMo-2-1124-13B-Instruct/MathCONTA_test_full_temp0_nstarts20_n1_2_3_5_7_seed42.json


In [ ]:
#!Parameter
parameter1=1
theta=0.325
exp_id_test=exp_id
method_name=method_name
metric_col=feature_col
log_path_base=log_path_base

ngram_df=get_ngram_df_from_generation_data(MathCONTA_ngram_data, n=parameter1)
len(ngram_df)

30

In [ ]:
test_dict=evaluate_method_standard(
    ngram_df,
    metric_col=metric_col,
    theta=theta,
    model_id=model_id,
    data_name=data_name,
    method_name=method_name,
    parameter1=parameter1,
    n_bootstrap=1000,
    seed=42
)
test_dict

{'model_id': 'allenai/OLMo-2-1124-13B-Instruct',
 'data_name': 'MathCONTA_test_full_temp0',
 'datetime': '2025-04-11T17:32:32.011931',
 'method_name': 'ngram_acc',
 'parameter': {'parameter1': 1, 'parameter2': None, 'theta': '0.325'},
 'metrics': {'accuracy': 0.5,
  'accuracy_95CI': (0.3325, 0.6674999999999992),
  'precision': 0.5,
  'recall': 1.0,
  'f1_score': 0.6666666666666666,
  'confusion_matrix': [[0, 15], [0, 15]],
  'mcnemar_b': 0,
  'mcnemar_c': 0,
  'mcnemar_p_value': 1.0},
 'n_bootstrap': 1000,
 'seed': 42}

In [ ]:
save_accuracy_log(log_path_base=log_path_base,model_id=model_id,method_name=method_name,exp_id=exp_id_test, data_name=data_name,out_dict=test_dict)

Accuracy log saved in /content/drive/MyDrive/Masterarbeit25/cdm_data/MathCONTA_v1/allenai/OLMo-2-1124-13B-Instruct/ngram_acc/MathCONTA_test_full_temp0_accuracylog_overleaf2.json


## RUN END2END

In [ ]:
n=5
theta=0.075

In [ ]:
run_ngram(
    model_id=model_id,
    filtered_ds=filtered_ds,
    model=model,
    tokenizer=tokenizer,
    data_name=data_name,
    force_reprocess=force_reprocess,
    n_starts=n_starts,
    n_range=n_range,
    temp=temp,
    torch_seed=torch_seed,
    seed_rand=seed_rand,
    start_offset=start_offset,
    random_starts=random_starts,
    only_problem=only_problem,
    verbose=verbose,
    dir_token_path=dir_token_path,
    n=n,
    theta=theta
)


File path: /content/drive/MyDrive/Masterarbeit25/MathCONTA_ngram_generation_data/deepseek-ai/deepseek-math-7b-instruct/AIME_full_temp0_nstarts20_n1_2_3_5_7_seed42.json
Loading existing n-gram generation data from: /content/drive/MyDrive/Masterarbeit25/MathCONTA_ngram_generation_data/deepseek-ai/deepseek-math-7b-instruct/AIME_full_temp0_nstarts20_n1_2_3_5_7_seed42.json


{'accuracy': 0.55,
 'precision': 0.5263157894736842,
 'recall': 1.0,
 'f1_score': 0.6896551724137931,
 'confusion_matrix': array([[ 1,  9],
        [ 0, 10]])}

## Combined Evaluation across LLMs

In [ ]:
model_ids = ["deepseek-ai/deepseek-math-7b-instruct",
             "EleutherAI/llemma_7b",
             "allenai/OLMo-7B-0724-Instruct-hf",
              "allenai/OLMo-2-1124-13B-Instruct"]
model_id_combi = "DS_LE_OL_OL2"

## TRAIN SET CV - Parameter Tuning

In [ ]:
data_name="MathCONTA_train_full_temp0" #please add full|problem and tempx to the name: e.g. AIME_full_temp0
n_starts=20
n_range=[1,2,3,5,7]
temp=0.0
torch_seed=42
seed_rand=42
start_offset=5
random_starts=True
only_problem=False
force_reprocess=False
verbose=False
data=ds_conta


MathCONTA_ngram_data_combined = []
for model_id in model_ids:
  MathCONTA_ngram_data=create_ngram_generation_data(model_id=model_id,ds_conta=data,
                                                  model=model, tokenizer=tokenizer, data_name=data_name,
                                                  force_reprocess=force_reprocess,n_starts=n_starts,n=n_range,temp=temp,
                                                  torch_seed=torch_seed,seed_rand=seed_rand,start_offset=start_offset,random_starts=random_starts,
                                                  only_problem=only_problem,verbose=verbose,
                                                  base_path_token=dir_token_path)
  MathCONTA_ngram_data_combined += MathCONTA_ngram_data

len(MathCONTA_ngram_data_combined)

File path: /content/drive/MyDrive/Masterarbeit25/MathCONTA_ngram_generation_data/deepseek-ai/deepseek-math-7b-instruct/MathCONTA_train_full_temp0_nstarts20_n1_2_3_5_7_seed42.json
Loading existing n-gram generation data from: /content/drive/MyDrive/Masterarbeit25/MathCONTA_ngram_generation_data/deepseek-ai/deepseek-math-7b-instruct/MathCONTA_train_full_temp0_nstarts20_n1_2_3_5_7_seed42.json
File path: /content/drive/MyDrive/Masterarbeit25/MathCONTA_ngram_generation_data/EleutherAI/llemma_7b/MathCONTA_train_full_temp0_nstarts20_n1_2_3_5_7_seed42.json
Loading existing n-gram generation data from: /content/drive/MyDrive/Masterarbeit25/MathCONTA_ngram_generation_data/EleutherAI/llemma_7b/MathCONTA_train_full_temp0_nstarts20_n1_2_3_5_7_seed42.json
File path: /content/drive/MyDrive/Masterarbeit25/MathCONTA_ngram_generation_data/allenai/OLMo-7B-0724-Instruct-hf/MathCONTA_train_full_temp0_nstarts20_n1_2_3_5_7_seed42.json
Loading existing n-gram generation data from: /content/drive/MyDrive/Maste

280

In [ ]:
#!Parameter 2D
data=MathCONTA_ngram_data_combined
feature_col="ngram_acc"
label_col="LABEL_BINARY"
n_range=n_range
cv_folds=5
cv_seed=42
log_path_base =DRIVE_PATH / "cdm_data" / "MathCONTA_v1"
exp_id="overleaf2combi"#+'_'+category
method_name="ngram_acc"
model_id = model_id_combi

In [ ]:
out_dict=tune_ngram_acc(ngram_data=data,feature_col=feature_col,
                          label_col=label_col,cv_folds=cv_folds,cv_seed=cv_seed,
                          n_range=n_range, model_id=model_id,data_name=data_name,
                          method_name=method_name,log_path_base=log_path_base,exp_id=exp_id)

Testing n=1 | median_threshold=0.700000 | mean_cvacc_test=0.603571
Testing n=2 | median_threshold=0.525000 | mean_cvacc_test=0.567857
Testing n=3 | median_threshold=0.375000 | mean_cvacc_test=0.535714
Testing n=5 | median_threshold=0.275000 | mean_cvacc_test=0.525000
Testing n=7 | median_threshold=0.125000 | mean_cvacc_test=0.471429
Best log entry:
{'parameter': {'n': 1}, 'global_threshold': 0.675, 'median_threshold': 0.7, 'mean_cvacc_train': 0.603571, 'mean_cvacc_test': 0.603571}
Accuracy log saved in /content/drive/MyDrive/Masterarbeit25/cdm_data/MathCONTA_v1/DS_LE_OL_OL2/ngram_acc/MathCONTA_train_full_temp0_accuracylog_overleaf2combi.json


## TEST SET EVALUATION

In [ ]:
data_name="MathCONTA_test_full_temp0"
MathCONTA_ngram_data_combined = []
for model_id in model_ids:
  MathCONTA_ngram_data=create_ngram_generation_data(model_id=model_id,ds_conta=data,
                                                  model=model, tokenizer=tokenizer, data_name=data_name,
                                                  force_reprocess=force_reprocess,n_starts=n_starts,n=n_range,temp=temp,
                                                  torch_seed=torch_seed,seed_rand=seed_rand,start_offset=start_offset,random_starts=random_starts,
                                                  only_problem=only_problem,verbose=verbose,
                                                  base_path_token=dir_token_path)
  MathCONTA_ngram_data_combined += MathCONTA_ngram_data

len(MathCONTA_ngram_data_combined)

File path: /content/drive/MyDrive/Masterarbeit25/MathCONTA_ngram_generation_data/deepseek-ai/deepseek-math-7b-instruct/MathCONTA_test_full_temp0_nstarts20_n1_2_3_5_7_seed42.json
Loading existing n-gram generation data from: /content/drive/MyDrive/Masterarbeit25/MathCONTA_ngram_generation_data/deepseek-ai/deepseek-math-7b-instruct/MathCONTA_test_full_temp0_nstarts20_n1_2_3_5_7_seed42.json
File path: /content/drive/MyDrive/Masterarbeit25/MathCONTA_ngram_generation_data/EleutherAI/llemma_7b/MathCONTA_test_full_temp0_nstarts20_n1_2_3_5_7_seed42.json
Loading existing n-gram generation data from: /content/drive/MyDrive/Masterarbeit25/MathCONTA_ngram_generation_data/EleutherAI/llemma_7b/MathCONTA_test_full_temp0_nstarts20_n1_2_3_5_7_seed42.json
File path: /content/drive/MyDrive/Masterarbeit25/MathCONTA_ngram_generation_data/allenai/OLMo-7B-0724-Instruct-hf/MathCONTA_test_full_temp0_nstarts20_n1_2_3_5_7_seed42.json
Loading existing n-gram generation data from: /content/drive/MyDrive/Masterarbe

120

In [ ]:
#!Parameter
parameter1=1
theta=0.675
exp_id_test=exp_id
method_name=method_name
metric_col=feature_col
log_path_base=log_path_base
model_id=model_id_combi

ngram_df=get_ngram_df_from_generation_data(MathCONTA_ngram_data_combined, n=parameter1)
len(ngram_df)

120

In [ ]:
test_dict=evaluate_method_standard(
    ngram_df,
    metric_col=metric_col,
    theta=theta,
    model_id=model_id,
    data_name=data_name,
    method_name=method_name,
    parameter1=parameter1,
    n_bootstrap=1000,
    seed=42
)
test_dict

{'model_id': 'DS_LE_OL_OL2',
 'data_name': 'MathCONTA_test_full_temp0',
 'datetime': '2025-04-12T18:47:43.987977',
 'method_name': 'ngram_acc',
 'parameter': {'parameter1': 1, 'parameter2': None, 'theta': '0.675'},
 'metrics': {'accuracy': 0.525,
  'accuracy_95CI': (0.43333333333333335, 0.6083333333333333),
  'precision': 0.5254237288135594,
  'recall': 0.5166666666666667,
  'f1_score': 0.5210084033613446,
  'confusion_matrix': [[32, 28], [29, 31]],
  'mcnemar_b': 32,
  'mcnemar_c': 29,
  'mcnemar_p_value': 0.7981526273057179},
 'n_bootstrap': 1000,
 'seed': 42}

In [ ]:
save_accuracy_log(log_path_base=log_path_base,model_id=model_id,method_name=method_name,exp_id=exp_id_test, data_name=data_name,out_dict=test_dict)

Accuracy log saved in /content/drive/MyDrive/Masterarbeit25/cdm_data/MathCONTA_v1/DS_LE_OL_OL2/ngram_acc/MathCONTA_test_full_temp0_accuracylog_overleaf2combi.json
